In [1]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [8]:


mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\IHCA'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    low_blue = np.array([55, 50, 50])
    high_blue = np.array([130, 255, 255])

    # Define the range of red color in HSV
    low_red = np.array([130, 50, 50])
    high_red = np.array([180, 255, 255])

    # Define HSV range for brown color
    low_brown = np.array([0, 20, 30])
    high_brown = np.array([22, 100, 240])

    # Create masks based on the color ranges
    blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    red_mask = cv2.inRange(hsv, low_red, high_red)
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    overlay[blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    overlay[red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)

    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    out_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt3'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    full_path = join(out_path, read_path)
    cv2.imwrite(full_path, masked_image)
    print(f"Saved processed image to {full_path}")  # Debugging line to confirm save

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\IHCA\6521-002_IHCA.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt3\6521-002_IHCA.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\IHCA\6521-006_IHCA.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt3\6521-006_IHCA.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\IHCA\6521-010_IHCA.tif


KeyboardInterrupt: 

In [8]:
def mouseRGB(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Checks mouse left button down condition
        colorsB = image[y, x, 0]
        colorsG = image[y, x, 1]
        colorsR = image[y, x, 2]
        colors = image[y, x]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        print("HSV :", hsv)
        print("Red: ", colorsR)
        print("Green: ", colorsG)
        print("Blue: ", colorsB)
        print("BRG Format: ", colors)
        print("Coordinates of pixel: X: ", x, "Y: ", y)

# Read an image
image_path = r"\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-004_IHCC"
image = cv2.imread(image_path, cv2.IMREAD_COLOR)

# Verify image loading
if image is None:
    print(f"Failed to load image at {image_path}")
else:
    # Print image dimensions
    print(f"Image loaded with dimensions: {image.shape}")

    # Create a window and bind the function to window
    cv2.namedWindow('mouseRGB', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    # Do until esc pressed
    while True:
        cv2.imshow('mouseRGB', image)
        if cv2.waitKey(20) & 0xFF == 27:  # ESC key to break
            break

    # If esc pressed, finish
    cv2.destroyAllWindows()

Failed to load image at \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-004_IHCC


In [6]:
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join

# Initialize global variables
zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
stain_colors = ["brown", "blue", "red"]
hsv_values = [[] for _ in stain_colors]  # List to store HSV values for each color
hsv_index = 0

def set_hsv_index(value):
    global hsv_index
    try:
        hsv_index = int(value)
        if 0 <= hsv_index < len(stain_colors):
            print(f"HSV index set to: {hsv_index} ({stain_colors[hsv_index]})")
        else:
            print(f"Invalid index. Please enter a number between 0 and {len(stain_colors) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

def mouseRGB(event, x, y, flags, param):
    global zoom_factor, pan_x, pan_y, cursor_x, cursor_y, hsv_values, hsv_index

    if event == cv2.EVENT_LBUTTONDOWN:  # Check mouse left button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        hsv = hsv[0][0]
        print("Left Click - HSV :", hsv)
        print("Coordinates of pixel: X: ", x, "Y: ", y)
        mouseRGB.last_x, mouseRGB.last_y = x, y

    elif event == cv2.EVENT_RBUTTONDOWN:  # Check mouse right button down condition
        colorsB = display_img[y, x, 0]
        colorsG = display_img[y, x, 1]
        colorsR = display_img[y, x, 2]
        hsv_value = np.uint8([[[colorsB, colorsG, colorsR]]])
        hsv = cv2.cvtColor(hsv_value, cv2.COLOR_BGR2HSV)
        hsv = hsv[0][0]
        print("Right Click - HSV :", hsv)
        print("Coordinates of pixel: X: ", x, "Y: ", y)
        mouseRGB.last_x, mouseRGB.last_y = x, y

        # Append HSV value to the list for the specified index
        hsv_values[hsv_index].append(hsv.tolist())

    elif event == cv2.EVENT_MOUSEWHEEL:
        cursor_x, cursor_y = x, y
        if flags > 0:
            zoom_factor += 0.1
        else:
            zoom_factor -= 0.1
        zoom_factor = max(0.1, zoom_factor)
        update_display_img()

    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        pan_x += x - mouseRGB.last_x
        pan_y += y - mouseRGB.last_y
        mouseRGB.last_x, mouseRGB.last_y = x, y
        update_display_img()

def update_display_img():
    global display_img, zoom_factor, pan_x, pan_y, cursor_x, cursor_y
    h, w = img.shape[:2]
    # Center of the zoom
    center_x, center_y = cursor_x, cursor_y
    M = np.float32([[zoom_factor, 0, center_x - center_x * zoom_factor + pan_x],
                    [0, zoom_factor, center_y - center_y * zoom_factor + pan_y]])
    display_img = cv2.warpAffine(img, M, (w, h))

def crop_center(image, crop_fraction=0.5):
    # Get the dimensions of the image
    h, w = image.shape[:2]

    # Calculate the coordinates for the center crop
    crop_w, crop_h = int(w * crop_fraction), int(h * crop_fraction)
    start_x = (w - crop_w) // 2
    start_y = (h - crop_h) // 2
    end_x = start_x + crop_w
    end_y = start_y + crop_h

    # Crop the image
    cropped_img = image[start_y:end_y, start_x:end_x]

    return cropped_img

mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
mouseRGB.last_x = 0
mouseRGB.last_y = 0

# Ask the user to input the HSV index
input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
set_hsv_index(input_value)

# Filter to get only files that end with 'IHCC.tif'
ihcc_files = [f for f in onlyfiles if f.endswith('IHCB.tif')]

for read_path in ihcc_files:
    final_path = os.path.join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue

    img = crop_center(img)
    display_img = img.copy()

    cv2.namedWindow('mouseRGB', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    while True:
        cv2.imshow('mouseRGB', display_img)
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC key to break
            break
        elif key == ord('x'):  # Press 'x' key to break the for loop
            break
        elif key == ord('c'):  # Press 'c' key to change HSV index
            # Ask the user to input the HSV index
            input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
            set_hsv_index(input_value)

    if key == ord('x'):  # Check if 'x' was pressed
        break  # Break out of the for loop

    cv2.destroyAllWindows()



HSV index set to: 0 (brown)
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-003_IHCB.tif
Left Click - HSV : [114   6 216]
Coordinates of pixel: X:  4929 Y:  4529
Left Click - HSV : [108   6 224]
Coordinates of pixel: X:  5146 Y:  4272
Left Click - HSV : [  0   0 223]
Coordinates of pixel: X:  6977 Y:  5895
Left Click - HSV : [110  14 214]
Coordinates of pixel: X:  3720 Y:  5104
Left Click - HSV : [108  20 226]
Coordinates of pixel: X:  7173 Y:  6436
Left Click - HSV : [114  11 231]
Coordinates of pixel: X:  3482 Y:  6351
Left Click - HSV : [109  12 236]
Coordinates of pixel: X:  7239 Y:  4932
Left Click - HSV : [111  24 212]
Coordinates of pixel: X:  5537 Y:  915
Left Click - HSV : [108   6 231]
Coordinates of pixel: X:  7657 Y:  3923
Left Click - HSV : [  0   1 230]
Coordinates of pixel: X:  6339 Y:  4864
Left Click - HSV : [107  10 223]
Coordinates of pixel: X:  2296 Y:  5932
Left Click - HSV : [116  16 222]
Coordinates of pixel: X:  8195 Y:  

In [3]:
# Initialize lists to store max and min values for each color category
max_values_list = []
min_values_list = []
# Calculate max and min values separately for each color category
for i in hsv_values:  # Iterating over each color category
    max_values = np.amax(i, axis=0).tolist()  # Convert to list for better readability
    min_values = np.amin(i, axis=0).tolist()  # Convert to list for better readability
    max_values_list.append(max_values)
    min_values_list.append(min_values)

print("Max values for each color category:", max_values_list)
print("Min values for each color category:", min_values_list)


Max values for each color category: [[17, 171, 143], [116, 181, 171], [171, 114, 152]]
Min values for each color category: [[4, 46, 70], [112, 89, 69], [139, 56, 78]]


In [7]:

mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
ihcc_files = [f for f in onlyfiles if f.endswith('IHCC.tif')]
for read_path in ihcc_files:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    #low_blue = np.array([55, 50, 50])
    #high_blue = np.array([130, 255, 255])
    low_blue = np.asarray(min_values_list[2])
    high_blue = np.asarray(max_values_list[2])


    # Define the range of red color in HSV
    #low_red = np.array([130, 50, 50])
    #high_red = np.array([180, 255, 255])
    low_red = np.asarray(min_values_list[1])
    high_red = np.asarray(max_values_list[1])

    # Define HSV range for brown color
    #low_brown = np.array([0, 50, 40])
    #high_brown = np.array([22, 100, 240])
    low_brown = np.asarray(min_values_list[0])
    high_brown = np.asarray(max_values_list[0])

    # Create masks based on the color ranges
    blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    red_mask = cv2.inRange(hsv, low_red, high_red)
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    overlay[blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    overlay[red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)

    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    out_path = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    full_path = join(out_path, read_path)
    cv2.imwrite(full_path, masked_image)
    print(f"Saved processed image to {full_path}")  # Debugging line to confirm save"""

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-004_IHCC.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-004_IHCC.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-008_IHCC.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-008_IHCC.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-012_IHCC.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-012_IHCC.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-016_IHCC.tif
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\attempt1\6521-016_IHCC.tif
Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\10x_v2\chop\6521-020_IHCC.tif
Saved processed image to \\10.17.97.73

In [3]:
mypath = r'\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for read_path in onlyfiles:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)
    cv2.namedWindow('mouseRGB')
    cv2.setMouseCallback('mouseRGB', mouseRGB)

    # Display the image in the window
    while True:
        cv2.imshow('mouseRGB', img)
        if cv2.waitKey(20) & 0xFF == 27:  # ESC key to break
            break

    # Close the window after ESC key is pressed
    cv2.destroyAllWindows()

Processing \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\2x_v2\chop\6521-001_HE.tif
Left Click - HSV : [[[111   8 211]]]
Red:  204
Green:  206
Blue:  211
BRG Format:  [211 206 204]
Coordinates of pixel: X:  3204 Y:  1757
Right Click - HSV : [[[150  41 198]]]
Red:  198
Green:  166
Blue:  198
BRG Format:  [198 166 198]
Coordinates of pixel: X:  3370 Y:  1736
Right Click - HSV : [[[155  72 159]]]
Red:  159
Green:  114
Blue:  152
BRG Format:  [152 114 159]
Coordinates of pixel: X:  3021 Y:  1703
Right Click - HSV : [[[149  76 161]]]
Red:  159
Green:  113
Blue:  161
BRG Format:  [161 113 159]
Coordinates of pixel: X:  2573 Y:  1328
Right Click - HSV : [[[162  32 221]]]
Red:  221
Green:  193
Blue:  210
BRG Format:  [210 193 221]
Coordinates of pixel: X:  2160 Y:  1195
Right Click - HSV : [[[137  86 140]]]
Red:  120
Green:  93
Blue:  140
BRG Format:  [140  93 120]
Coordinates of pixel: X:  2133 Y:  1682
Left Click - HSV : [[[154  61 171]]]
Red:  171
Green:  130
Blue:  165
BRG F

KeyboardInterrupt: 